# Proofreading Script

## Libraries

In [135]:
# Import the libraries
import pandas as pd
import math
import re

## Data

In [136]:
# Path to the Excel file
file_path = "/Users/merveogretmek/Desktop/AL/March/14th/Proofreading/CT-BA-25-001_3132025.xlsx"

# Load the Excel file into a DataFrame
df = pd.read_excel(file_path, skiprows=[1,2])

# Set display option to show full text in columns
pd.set_option('display.max_colwidth', None)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


## Functions

### Custom Rounding Functions

In [137]:
# --- Custom Rounding for Seat Height (for Product Name & Description) ---
def custom_round_seat_height(attr_value: str) -> str:
    """
    For the Product Name and Description checks:
      - If the value is between 25 and 26 (exclusive), round up to 26.
      - If the value is between 30 and 31 (exclusive), round down to 30.
      - Otherwise, use standard rounding (nearest integer).
    Returns the rounded value as a string.
    """
    try:
        val = float(attr_value)
    except ValueError:
        return attr_value.lower()  # fallback if conversion fails
    if 25 < val < 26:
        normalized_val = 26
    elif 30 < val < 31:
        normalized_val = 30
    else:
        normalized_val = int(round(val))
    return str(normalized_val)

In [138]:
# --- Custom Rounding for Dimensions (for Bullet Feature) ---
def custom_round_dimension(val: float) -> int:
    """
    For dimension values:
      - If the value is between 39 and 40, always round up to 40.
      - If the value is between 19 and 20, always round up to 20.
      - Otherwise, use standard mathematical rounding.
    """
    if 39 < val < 40:
        return 40
    elif 19 < val < 20:
        return 20
    else:
        return int(round(val))

### Product Name Check

In [139]:
# --- Product Name Check ---
prod_attributes = [
    "Collection Name",
    "Seat Height 1",
    "Finish Color",
    "Manufactured Woods Use",
    "General Category",
    "Upholstery Color"
]

def check_product_name(row):
    """
    Verify that each attribute's value appears in the 'Individual Product Name'.
    For 'Seat Height 1', the custom rounding is applied.
    """
    product_name = str(row["Individual Product Name"]).lower() if pd.notna(row["Individual Product Name"]) else ""
    mismatches = {}
    for attr in prod_attributes:
        attr_value = str(row[attr]) if pd.notna(row[attr]) else ""
        if not attr_value:
            continue
        if attr == "Seat Height 1":
            normalized_attr = custom_round_seat_height(attr_value)
            if normalized_attr not in product_name:
                mismatches[attr] = attr_value
        else:
            if attr_value.lower() not in product_name:
                mismatches[attr] = attr_value
    return mismatches

df["Product_Name_Mismatches"] = df.apply(check_product_name, axis=1)

### Description Check

In [140]:
# --- Description Check ---
desc_attributes = [
    "Collection Name",
    "Seat Height 1",
    "General Category",
    "Upholstery Color",
    "Finish Color",
    "Manufactured Woods Use"
]

def check_description(row):
    """
    Verify that each attribute's value appears in the 'Description' column.
    For 'Seat Height 1', the custom rounding is applied.
    """
    description = str(row["Description"]).lower() if pd.notna(row["Description"]) else ""
    mismatches = {}
    for attr in desc_attributes:
        attr_value = str(row[attr]) if pd.notna(row[attr]) else ""
        if not attr_value:
            continue
        if attr == "Seat Height 1":
            normalized_attr = custom_round_seat_height(attr_value)
            if normalized_attr not in description:
                mismatches[attr] = attr_value
        else:
            if attr_value.lower() not in description:
                mismatches[attr] = attr_value
    return mismatches

df["Description_Mismatches"] = df.apply(check_description, axis=1)

### Tolerance (<=1) 

In [141]:
# --- Helper Function for Comparing Numeric Values with Tolerance ---
def compare_with_tolerance(bullet_val, exp_val, tol = 1):
    """
    Compares two numeric values (given as strings) after converting to integers.
    Returns a mismatch message if the absolute difference is greater than tol;
    otherwise returns None.
    If conversion fails, falls back to a simple string comparison.
    """
    try:
        bullet_num = int(bullet_val)
        exp_num = int(exp_val)
        if abs(bullet_num - exp_num) > tol:
            return f"Bullet: {bullet_val} vs Expected: {exp_val}"
    except Exception:
        if bullet_val != exp_val:
            return f"Bullet: {bullet_val} vs Expected: {exp_val}"
    
    return None

### Bullet Features Check

In [ ]:
# -- Parse Bullet Feature 1 --
def parse_bullet_feature(bullet_str):
    """
    Parse the Bullet Feature 1 string into its parts:
      - Dimension: Depth, Width, Height (expected format:
        Dimension: {depth}" x {width}" x {height}")
      - Seat Height (expected format: Seat Height: {seat}")
    Returns a dictionary with keys:
      Bullet_Depth, Bullet_Width, Bullet_Height, Bullet_Seat_Height.
    The parsing is done case-insensitively.
    """
    if not bullet_str or not isinstance(bullet_str, str):
        return {"Bullet_Depth": "", "Bullet_Width": "", "Bullet_Height": "", "Bullet_Seat_Height": ""}
    bullet_str_lower = bullet_str.lower()
    m = re.search(r'dimension:\s*(\d+)"\s*x\s*(\d+)"\s*x\s*(\d+)"', bullet_str_lower)
    if m:
        bullet_depth = m.group(1)
        bullet_width = m.group(2)
        bullet_height = m.group(3)
    else:
        bullet_depth = bullet_width = bullet_height = ""
    m_seat = re.search(r'seat height:\s*(\d+)"', bullet_str_lower)
    bullet_seat = m_seat.group(1) if m_seat else ""
    return {"Bullet_Depth": bullet_depth, "Bullet_Width": bullet_width,
            "Bullet_Height": bullet_height, "Bullet_Seat_Height": bullet_seat}

# Create temporary columns for Bullet Feature 1 by parsing the text
bf1 = df["Bullet Feature 1"].apply(parse_bullet_feature)
bf1_df = pd.DataFrame(bf1.tolist())
df = pd.concat([df, bf1_df], axis=1)

def check_bullet_feature_1_details(row):
    """
    Compare each part of Bullet Feature 1 (dimension and seat height)
    with the expected values computed from the original columns.
    Expected:
      - Depth, Width, Height: computed from 'Product Depth 1', 'Product Width 1', 'Product Height 1'
        using custom_round_dimension.
      - Seat Height: computed from 'Seat Height 1' using standard rounding.
    A mismatch is flagged only if the numeric difference exceeds 1.
    """
    try:
        exp_depth = str(custom_round_dimension(float(row["Product Depth 1"])))
    except (ValueError, TypeError):
        exp_depth = ""
    try:
        exp_width = str(custom_round_dimension(float(row["Product Width 1"])))
    except (ValueError, TypeError):
        exp_width = ""
    try:
        exp_height = str(custom_round_dimension(float(row["Product Height 1"])))
    except (ValueError, TypeError):
        exp_height = ""
    try:
        exp_seat = str(int(round(float(row["Seat Height 1"]))))
    except (ValueError, TypeError):
        exp_seat = ""
    
    bullet_depth = str(row.get("Bullet_Depth", ""))
    bullet_width = str(row.get("Bullet_Width", ""))
    bullet_height = str(row.get("Bullet_Height", ""))
    bullet_seat = str(row.get("Bullet_Seat_Height", ""))
    
    details = {}
    diff = compare_with_tolerance(bullet_depth, exp_depth, tol=1)
    if diff:
        details['Depth'] = diff
    diff = compare_with_tolerance(bullet_width, exp_width, tol=1)
    if diff:
        details['Width'] = diff
    diff = compare_with_tolerance(bullet_height, exp_height, tol=1)
    if diff:
        details['Height'] = diff
    diff = compare_with_tolerance(bullet_seat, exp_seat, tol=1)
    if diff:
        details['Seat Height'] = diff
        
    return details

df["Bullet_Feature_1_Details"] = df.apply(check_bullet_feature_1_details, axis=1)

In [ ]:
# -- Bullet Feature 2 Check --
def check_bullet_feature_2(row):
    """
    For Bullet Feature 2:
      Expected attribute: Manufactured Woods Use.
      Verify (case insensitively) that the Bullet Feature 2 text contains the
      value from 'Manufactured Woods Use'.
    """
    bullet2 = str(row["Bullet Feature 2"]).lower() if pd.notna(row["Bullet Feature 2"]) else ""
    mwoods = str(row["Manufactured Woods Use"]).lower() if pd.notna(row["Manufactured Woods Use"]) else ""
    mismatches = {}
    if mwoods and mwoods not in bullet2:
        mismatches["Manufactured Woods Use"] = mwoods
    return mismatches

df["Bullet_Feature_2_Details"] = df.apply(check_bullet_feature_2, axis=1)

In [144]:
# -- Bullet Feature 3 Check --
def check_bullet_feature_3(row):
    """
    For Bullet Feature 3:
      Expected attributes:
        - Back Height from "Back Height 1"
        - Back Style from "Back Style"
        - Style: check that at least one of the semicolon-separated values from 
                 "Style" appears in the bullet.
    """
    bullet3 = str(row["Bullet Feature 3"]).lower() if pd.notna(row["Bullet Feature 3"]) else ""
    mismatches = {}
    
    # Back Height check
    back_height = str(row["Back Height 1"]).lower() if pd.notna(row["Back Height 1"]) else ""
    if back_height and back_height not in bullet3:
        mismatches["Back Height 1"] = back_height
    
    # Back Style check
    back_style = str(row["Back Style"]).lower() if pd.notna(row["Back Style"]) else ""
    if back_style and back_style not in bullet3:
        mismatches["Back Style"] = back_style
    
    # Style check (multiple expected values)
    style_raw = str(row["Style"]).lower() if pd.notna(row["Style"]) else ""
    if style_raw:
        expected_styles = [s.strip() for s in style_raw.split(';')]
        if not any(s in bullet3 for s in expected_styles):
            mismatches["Style"] = style_raw
    
    return mismatches

df["Bullet_Feature_3_Details"] = df.apply(check_bullet_feature_3, axis=1)

In [145]:
# -- Bullet Feature 5 Check --
def check_bullet_feature_5(row):
    """
    For Bullet Feature 5:
      Expected attributes:
        - Seat Shape from "Seat Shape"
        - Cushion Fill Density from "Cushion Fill Density"
        - Cushion Fill Material from "Cushion Fill Material"
        - Upholstery Material from "Upholstery Material"
    Verify that each expected value appears in Bullet Feature 5 text.
    """
    bullet5 = str(row["Bullet Feature 5"]).lower() if pd.notna(row["Bullet Feature 5"]) else ""
    mismatches = {}
    seat_shape = str(row["Seat Shape"]).lower() if pd.notna(row["Seat Shape"]) else ""
    if seat_shape and seat_shape not in bullet5:
        mismatches["Seat Shape"] = seat_shape
    cushion_density = str(row["Cushion Fill Density"]).lower() if pd.notna(row["Cushion Fill Density"]) else ""
    if cushion_density and cushion_density not in bullet5:
        mismatches["Cushion Fill Density"] = cushion_density
    cushion_material = str(row["Cushion Fill Material"]).lower() if pd.notna(row["Cushion Fill Material"]) else ""
    if cushion_material and cushion_material not in bullet5:
        mismatches["Cushion Fill Material"] = cushion_material
    upholstery_material = str(row["Upholstery Material"]).lower() if pd.notna(row["Upholstery Material"]) else ""
    if upholstery_material and upholstery_material not in bullet5:
        mismatches["Upholstery Material"] = upholstery_material
    return mismatches

df["Bullet_Feature_5_Details"] = df.apply(check_bullet_feature_5, axis=1)

In [146]:
# -- Bullet Feature 6 Check --
def check_bullet_feature_6(row):
    """
    For Bullet Feature 6:
      Expected attribute: Weight Capacity from "Weight Capacity 1".
      Verify that the Bullet Feature 6 text contains the value from 'Weight Capacity 1'.
    """
    bullet6 = str(row["Bullet Feature 6"]).lower() if pd.notna(row["Bullet Feature 6"]) else ""
    weight_cap = str(row["Weight Capacity 1"]).lower() if pd.notna(row["Weight Capacity 1"]) else ""
    mismatches = {}
    if weight_cap and weight_cap not in bullet6:
        mismatches["Weight Capacity"] = weight_cap
    return mismatches

df["Bullet_Feature_6_Details"] = df.apply(check_bullet_feature_6, axis=1)

## Mismatch Results

In [147]:
# -- Product Name Mismatches --
print("Product Name mismatches:")
print(df[["Individual Product Name", "Product_Name_Mismatches"]][df["Product_Name_Mismatches"].apply(lambda x: len(x) > 0)])

# -- Description Mismatches --
print("\nDescription mismatches:")
print(df[["Description", "Description_Mismatches"]][df["Description_Mismatches"].apply(lambda x: len(x) > 0)])

# -- Bullet Feature 1 Mismatches --
print("\nBullet Feature 1 Breakdown Mismatches:")
bf1_mismatch = df[df["Bullet_Feature_1_Details"].apply(lambda x: len(x) > 0)]
print(bf1_mismatch[["Bullet Feature 1", "Bullet_Feature_1_Details"]])

# -- Bullet Feature 2 Mismatches --
print("\nBullet Feature 2 Mismatches:")
bf2_mismatch = df[df["Bullet_Feature_2_Details"].apply(lambda x: len(x) > 0)]
print(bf2_mismatch[["Bullet Feature 2", "Bullet_Feature_2_Details"]])

# -- Bullet Feature 3 Mismatches --
print("\nBullet Feature 3 Mismatches:")
bf3_mismatch = df[df["Bullet_Feature_3_Details"].apply(lambda x: len(x) > 0)]
print(bf3_mismatch[["Bullet Feature 3", "Bullet_Feature_3_Details"]])

# -- Bullet Feature 5 Mismatches --
print("\nBullet Feature 5 Mismatches:")
bf5_mismatch = df[df["Bullet_Feature_5_Details"].apply(lambda x: len(x) > 0)]
print(bf5_mismatch[["Bullet Feature 5", "Bullet_Feature_5_Details"]])

# -- Bullet Feature 6 Mismatches --
print("\nBullet Feature 6 Mismatches:")
bf6_mismatch = df[df["Bullet_Feature_6_Details"].apply(lambda x: len(x) > 0)]
print(bf6_mismatch[["Bullet Feature 6", "Bullet_Feature_6_Details"]])

Product Name mismatches:
                                                 Individual Product Name  \
4  Elle 26 in. Counter Height Rubberwood Barstool with Ivory Fabric Seat   

          Product_Name_Mismatches  
4  {'Finish Color': 'Amber Glow'}  

Description mismatches:
Empty DataFrame
Columns: [Description, Description_Mismatches]
Index: []

Bullet Feature 1 Breakdown Mismatches:
                                Bullet Feature 1  \
4  Dimension: 22" x 21" x 33" - Seat Height: 26"   

                                                                                                 Bullet_Feature_1_Details  
4  {'Depth': 'Bullet: 22 vs Expected: 20', 'Width': 'Bullet: 21 vs Expected: 18', 'Height': 'Bullet: 33 vs Expected: 40'}  

Bullet Feature 2 Mismatches:
Empty DataFrame
Columns: [Bullet Feature 2, Bullet_Feature_2_Details]
Index: []

Bullet Feature 3 Mismatches:
                                                                                                                       

In [149]:
import pandas as pd
import math
import re

# =============================================================================
# 1. Load Data and Configure Display
# =============================================================================

# Path to your Excel file
file_path = "/Users/merveogretmek/Desktop/AL/March/14th/Proofreading/CT-BA-25-001_3132025.xlsx"

# Load the Excel file into a DataFrame (skip rows as needed)
df = pd.read_excel(file_path, skiprows=[1,2])

# Configure Pandas to display full text in columns (prevents truncation)
pd.set_option('display.max_colwidth', None)

# =============================================================================
# 2. Custom Rounding Functions
# =============================================================================

# -- Custom Rounding for Seat Height (for Product Name & Description) --
def custom_round_seat_height(attr_value: str) -> str:
    """
    For the Product Name and Description checks:
      - If the value is between 25 and 26 (exclusive), round up to 26.
      - If the value is between 30 and 31 (exclusive), round down to 30.
      - Otherwise, use standard rounding (nearest integer).
    Returns the rounded value as a string.
    """
    try:
        val = float(attr_value)
    except ValueError:
        return attr_value.lower()  # fallback if conversion fails
    if 25 < val < 26:
        normalized_val = 26
    elif 30 < val < 31:
        normalized_val = 30
    else:
        normalized_val = int(round(val))
    return str(normalized_val)

# -- Custom Rounding for Dimensions (for Bullet Feature 1) --
def custom_round_dimension(val: float) -> int:
    """
    For dimension values:
      - If the value is between 39 and 40, always round up to 40.
      - If the value is between 19 and 20, always round up to 20.
      - Otherwise, use standard mathematical rounding.
    """
    if 39 < val < 40:
        return 40
    elif 19 < val < 20:
        return 20
    else:
        return int(round(val))

# =============================================================================
# 3. Product Name and Description Checks (Case Insensitive)
# =============================================================================

# -- Product Name Check --
prod_attributes = [
    "Collection Name",
    "Seat Height 1",
    "Finish Color",
    "Manufactured Woods Use",
    "General Category",
    "Upholstery Color"
]

def check_product_name(row):
    """
    Verify that each attribute's value appears in the 'Individual Product Name'.
    Text is compared in lowercase for case-insensitive matching.
    For 'Seat Height 1', apply custom rounding.
    """
    product_name = str(row["Individual Product Name"]).lower() if pd.notna(row["Individual Product Name"]) else ""
    mismatches = {}
    for attr in prod_attributes:
        attr_value = str(row[attr]).lower() if pd.notna(row[attr]) else ""
        if not attr_value:
            continue
        if attr == "Seat Height 1":
            normalized_attr = custom_round_seat_height(row[attr])
            if normalized_attr.lower() not in product_name:
                mismatches[attr] = row[attr]
        else:
            if attr_value not in product_name:
                mismatches[attr] = row[attr]
    return mismatches

df["Product_Name_Mismatches"] = df.apply(check_product_name, axis=1)

# -- Description Check --
desc_attributes = [
    "Collection Name",
    "Seat Height 1",
    "General Category",
    "Upholstery Color",
    "Finish Color",
    "Manufactured Woods Use"
]

def check_description(row):
    """
    Verify that each attribute's value appears in the 'Description' column.
    Both the description text and the attribute value are compared in lowercase.
    For 'Seat Height 1', apply custom rounding.
    """
    description = str(row["Description"]).lower() if pd.notna(row["Description"]) else ""
    mismatches = {}
    for attr in desc_attributes:
        attr_value = str(row[attr]).lower() if pd.notna(row[attr]) else ""
        if not attr_value:
            continue
        if attr == "Seat Height 1":
            normalized_attr = custom_round_seat_height(row[attr])
            if normalized_attr.lower() not in description:
                mismatches[attr] = row[attr]
        else:
            if attr_value not in description:
                mismatches[attr] = row[attr]
    return mismatches

df["Description_Mismatches"] = df.apply(check_description, axis=1)

# =============================================================================
# 4. Bullet Feature 1 Check (Dimension & Seat Height) with Numeric Tolerance
# =============================================================================

# -- Helper Function: Compare Numeric Values with Tolerance --
def compare_with_tolerance(bullet_val, exp_val, tol=1):
    """
    Compare two numeric values (provided as strings) after converting to integers.
    Returns a mismatch message if the absolute difference is greater than tol;
    otherwise returns None.
    If conversion fails, a simple string comparison is performed.
    """
    try:
        bullet_num = int(bullet_val)
        exp_num = int(exp_val)
        if abs(bullet_num - exp_num) > tol:
            return f"Bullet: {bullet_val} vs Expected: {exp_val}"
    except Exception:
        if bullet_val != exp_val:
            return f"Bullet: {bullet_val} vs Expected: {exp_val}"
    return None

# -- Parse Bullet Feature 1 --
def parse_bullet_feature(bullet_str):
    """
    Parse the Bullet Feature 1 string into its parts:
      - Dimension: Depth, Width, Height (expected format:
        Dimension: {depth}" x {width}" x {height}")
      - Seat Height (expected format: Seat Height: {seat}")
    Returns a dictionary with keys:
      Bullet_Depth, Bullet_Width, Bullet_Height, Bullet_Seat_Height.
    Parsing is done case insensitively.
    """
    if not bullet_str or not isinstance(bullet_str, str):
        return {"Bullet_Depth": "", "Bullet_Width": "", "Bullet_Height": "", "Bullet_Seat_Height": ""}
    bullet_str_lower = bullet_str.lower()
    m = re.search(r'dimension:\s*(\d+)"\s*x\s*(\d+)"\s*x\s*(\d+)"', bullet_str_lower)
    if m:
        bullet_depth = m.group(1)
        bullet_width = m.group(2)
        bullet_height = m.group(3)
    else:
        bullet_depth = bullet_width = bullet_height = ""
    m_seat = re.search(r'seat height:\s*(\d+)"', bullet_str_lower)
    bullet_seat = m_seat.group(1) if m_seat else ""
    return {"Bullet_Depth": bullet_depth, "Bullet_Width": bullet_width,
            "Bullet_Height": bullet_height, "Bullet_Seat_Height": bullet_seat}

# Create temporary columns for Bullet Feature 1 by parsing the text
bf1 = df["Bullet Feature 1"].apply(parse_bullet_feature)
bf1_df = pd.DataFrame(bf1.tolist())
df = pd.concat([df, bf1_df], axis=1)

def check_bullet_feature_1_details(row):
    """
    Compare each part of Bullet Feature 1 (dimension and seat height) with the expected values
    computed from the original columns.
    Expected values:
      - Depth, Width, Height: computed from 'Product Depth 1', 'Product Width 1', 'Product Height 1'
        using custom_round_dimension.
      - Seat Height: computed from 'Seat Height 1' using standard rounding.
    A mismatch is flagged only if the numeric difference exceeds 1.
    """
    try:
        exp_depth = str(custom_round_dimension(float(row["Product Depth 1"])))
    except (ValueError, TypeError):
        exp_depth = ""
    try:
        exp_width = str(custom_round_dimension(float(row["Product Width 1"])))
    except (ValueError, TypeError):
        exp_width = ""
    try:
        exp_height = str(custom_round_dimension(float(row["Product Height 1"])))
    except (ValueError, TypeError):
        exp_height = ""
    try:
        exp_seat = str(int(round(float(row["Seat Height 1"]))))
    except (ValueError, TypeError):
        exp_seat = ""
    
    bullet_depth = str(row.get("Bullet_Depth", ""))
    bullet_width = str(row.get("Bullet_Width", ""))
    bullet_height = str(row.get("Bullet_Height", ""))
    bullet_seat = str(row.get("Bullet_Seat_Height", ""))
    
    details = {}
    diff = compare_with_tolerance(bullet_depth, exp_depth, tol=1)
    if diff:
        details['Depth'] = diff
    diff = compare_with_tolerance(bullet_width, exp_width, tol=1)
    if diff:
        details['Width'] = diff
    diff = compare_with_tolerance(bullet_height, exp_height, tol=1)
    if diff:
        details['Height'] = diff
    diff = compare_with_tolerance(bullet_seat, exp_seat, tol=1)
    if diff:
        details['Seat Height'] = diff
        
    return details

df["Bullet_Feature_1_Details"] = df.apply(check_bullet_feature_1_details, axis=1)

# =============================================================================
# 5. Additional Bullet Feature Checks (Case Insensitive)
# =============================================================================

# -- Bullet Feature 2 Check --
def check_bullet_feature_2(row):
    """
    For Bullet Feature 2:
      Expected attribute: Manufactured Woods Use.
      Verify (case insensitively) that the Bullet Feature 2 text contains the
      value from 'Manufactured Woods Use'.
    """
    bullet2 = str(row["Bullet Feature 2"]).lower() if pd.notna(row["Bullet Feature 2"]) else ""
    mwoods = str(row["Manufactured Woods Use"]).lower() if pd.notna(row["Manufactured Woods Use"]) else ""
    mismatches = {}
    if mwoods and mwoods not in bullet2:
        mismatches["Manufactured Woods Use"] = mwoods
    return mismatches

df["Bullet_Feature_2_Details"] = df.apply(check_bullet_feature_2, axis=1)

# -- Bullet Feature 3 Check --
def check_bullet_feature_3(row):
    """
    For Bullet Feature 3:
      Expected attributes:
        - Back Height from column "Back Height1 "
        - Back Style from column "Back Style"
        - Style from column "Style": at least one of the semicolon-separated values must appear.
      All comparisons are done case insensitively.
    """
    bullet3 = str(row["Bullet Feature 3"]).lower() if pd.notna(row["Bullet Feature 3"]) else ""
    mismatches = {}
    
    back_height = str(row["Back Height 1"]).lower() if pd.notna(row["Back Height 1"]) else ""
    if back_height and back_height not in bullet3:
        mismatches["Back Height 1"] = back_height
    
    back_style = str(row["Back Style"]).lower() if pd.notna(row["Back Style"]) else ""
    if back_style and back_style not in bullet3:
        mismatches["Back Style"] = back_style
    
    style_raw = str(row["Style"]).lower() if pd.notna(row["Style"]) else ""
    if style_raw:
        expected_styles = [s.strip() for s in style_raw.split(';')]
        if not any(s in bullet3 for s in expected_styles):
            mismatches["Style"] = style_raw
    
    return mismatches

df["Bullet_Feature_3_Details"] = df.apply(check_bullet_feature_3, axis=1)

# -- Bullet Feature 5 Check --
def check_bullet_feature_5(row):
    """
    For Bullet Feature 5:
      Expected attributes:
        - Seat Shape from "Seat Shape"
        - Cushion Fill Density from "Cushion Fill Density"
        - Cushion Fill Material from "Cushion Fill Material"
        - Upholstery Material from "Upholstery Material"
      Verify (case insensitively) that each expected value exists in the Bullet Feature 5 text.
    """
    bullet5 = str(row["Bullet Feature 5"]).lower() if pd.notna(row["Bullet Feature 5"]) else ""
    mismatches = {}
    seat_shape = str(row["Seat Shape"]).lower() if pd.notna(row["Seat Shape"]) else ""
    if seat_shape and seat_shape not in bullet5:
        mismatches["Seat Shape"] = seat_shape
    cushion_density = str(row["Cushion Fill Density"]).lower() if pd.notna(row["Cushion Fill Density"]) else ""
    if cushion_density and cushion_density not in bullet5:
        mismatches["Cushion Fill Density"] = cushion_density
    cushion_material = str(row["Cushion Fill Material"]).lower() if pd.notna(row["Cushion Fill Material"]) else ""
    if cushion_material and cushion_material not in bullet5:
        mismatches["Cushion Fill Material"] = cushion_material
    upholstery_material = str(row["Upholstery Material"]).lower() if pd.notna(row["Upholstery Material"]) else ""
    if upholstery_material and upholstery_material not in bullet5:
        mismatches["Upholstery Material"] = upholstery_material
    return mismatches

df["Bullet_Feature_5_Details"] = df.apply(check_bullet_feature_5, axis=1)

# -- Bullet Feature 6 Check --
def check_bullet_feature_6(row):
    """
    For Bullet Feature 6:
      Expected attribute: Weight Capacity from "Weight Capacity 1".
      Verify (case insensitively) that the Bullet Feature 6 text contains the value from 'Weight Capacity 1'.
    """
    bullet6 = str(row["Bullet Feature 6"]).lower() if pd.notna(row["Bullet Feature 6"]) else ""
    weight_cap = str(row["Weight Capacity 1"]).lower() if pd.notna(row["Weight Capacity 1"]) else ""
    mismatches = {}
    if weight_cap and weight_cap not in bullet6:
        mismatches["Weight Capacity"] = weight_cap
    return mismatches

df["Bullet_Feature_6_Details"] = df.apply(check_bullet_feature_6, axis=1)

# =============================================================================
# 6. Display Mismatch Results
# =============================================================================

# -- Product Name Mismatches --
print("Product Name mismatches:")
print(df[["Individual Product Name", "Product_Name_Mismatches"]][df["Product_Name_Mismatches"].apply(lambda x: len(x) > 0)])

# -- Description Mismatches --
print("\nDescription mismatches:")
print(df[["Description", "Description_Mismatches"]][df["Description_Mismatches"].apply(lambda x: len(x) > 0)])

# -- Bullet Feature 1 Mismatches --
print("\nBullet Feature 1 Breakdown Mismatches:")
bf1_mismatch = df[df["Bullet_Feature_1_Details"].apply(lambda x: len(x) > 0)]
print(bf1_mismatch[["Bullet Feature 1", "Bullet_Feature_1_Details"]])

# -- Bullet Feature 2 Mismatches --
print("\nBullet Feature 2 Mismatches:")
bf2_mismatch = df[df["Bullet_Feature_2_Details"].apply(lambda x: len(x) > 0)]
print(bf2_mismatch[["Bullet Feature 2", "Bullet_Feature_2_Details"]])

# -- Bullet Feature 3 Mismatches --
print("\nBullet Feature 3 Mismatches:")
bf3_mismatch = df[df["Bullet_Feature_3_Details"].apply(lambda x: len(x) > 0)]
print(bf3_mismatch[["Bullet Feature 3", "Bullet_Feature_3_Details"]])

# -- Bullet Feature 5 Mismatches --
print("\nBullet Feature 5 Mismatches:")
bf5_mismatch = df[df["Bullet_Feature_5_Details"].apply(lambda x: len(x) > 0)]
print(bf5_mismatch[["Bullet Feature 5", "Bullet_Feature_5_Details"]])

# -- Bullet Feature 6 Mismatches --
print("\nBullet Feature 6 Mismatches:")
bf6_mismatch = df[df["Bullet_Feature_6_Details"].apply(lambda x: len(x) > 0)]
print(bf6_mismatch[["Bullet Feature 6", "Bullet_Feature_6_Details"]])

Product Name mismatches:
                                                 Individual Product Name  \
4  Elle 26 in. Counter Height Rubberwood Barstool with Ivory Fabric Seat   

          Product_Name_Mismatches  
4  {'Finish Color': 'Amber Glow'}  

Description mismatches:
Empty DataFrame
Columns: [Description, Description_Mismatches]
Index: []

Bullet Feature 1 Breakdown Mismatches:
                                Bullet Feature 1  \
4  Dimension: 22" x 21" x 33" - Seat Height: 26"   

                                                                                                 Bullet_Feature_1_Details  
4  {'Depth': 'Bullet: 22 vs Expected: 20', 'Width': 'Bullet: 21 vs Expected: 18', 'Height': 'Bullet: 33 vs Expected: 40'}  

Bullet Feature 2 Mismatches:
Empty DataFrame
Columns: [Bullet Feature 2, Bullet_Feature_2_Details]
Index: []

Bullet Feature 3 Mismatches:
                                                                                                                       

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
